In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
new_dir = os.path.join("drive","MyDrive","chatbot")
os.chdir(new_dir)

In [23]:
!pip install -q datasets loralib sentencepiece bert-score
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.35.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.0 MB/s eta 0:00:00


In [3]:
! pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00


In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from tqdm import tqdm
from torch.utils.data import DataLoader
tqdm.pandas()

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

**Initialization of the model and tokenizer**

In [7]:
# Load the model and tokenizer
model_path = 'bioGPT'
tokenizer = AutoTokenizer.from_pretrained('microsoft/biogpt')
model= AutoModelForCausalLM.from_pretrained(model_path).to(device)
# Setting the padding token
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model

#Settings for A100 - For 3090
MICRO_BATCH_SIZE = 8  # change to 4 for 3090
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 2  # paper uses 3
LEARNING_RATE = 2e-5
CUTOFF_LEN = 256
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

model = prepare_model_for_int8_training(model, use_gradient_checkpointing=True)

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

**Data Preprocessing**

In [8]:
medical_data = pd.read_json("datasets/final_qa_dataset.json")
medical_data = medical_data.sample(n=150000)

In [9]:
medical_data['combined'] = "In the context of biomedicine and as a biomedicine specialist, " + medical_data['question'] +" " + medical_data['answer']

In [10]:
display(medical_data)

,question,answer,combined
3349,What are the treatments for mucopolysaccharido...,These resources address the diagnosis or manag...,In the context of biomedicine and as a biomedi...
147798,Does chronic nerve compression alter Schwann c...,Chronic entrapment injuries like carpal tunnel...,In the context of biomedicine and as a biomedi...
98482,Does stereotactic coordinate for intracerebrov...,P-MCAO causes relatively late but predictable ...,In the context of biomedicine and as a biomedi...
5657,What is (are) Duane syndrome type 2 ?,Duane syndrome is a disorder of eye movement. ...,In the context of biomedicine and as a biomedi...
133113,Does age range for inclusion affect ascertainm...,This study highlights the value of birth defec...,In the context of biomedicine and as a biomedi...
...,...,...,...
195460,Does effect of a risk-based multifactorial fal...,The program was not effective in reducing fall...,In the context of biomedicine and as a biomedi...
227247,Does interleukin 28B polymorphism predict pegy...,The IL28B rs12979860 SNP may have an added val...,In the context of biomedicine and as a biomedi...
89274,Is serum creatinine an inadequate screening te...,This study documents the substantial prevalenc...,In the context of biomedicine and as a biomedi...
225854,Does anticancer agent 3-bromopyruvic acid form...,3-BP reacts with GSH in the absence of cells a...,In the context of biomedicine and as a biomedi...


In [11]:
# Function to count words in a text
def word_count(text):
    return len(text.split(' '))

# Create a new column for word count
medical_data['word_count'] = medical_data['combined'].apply(word_count)

**Keep samples with length < 500**

In [11]:
medical_data = medical_data[medical_data['word_count'] < 500]

**Splitting the dataframe into training and validation and test**

In [12]:
# Split the dataset
train_med, test_med = train_test_split(medical_data, test_size=0.2)
val_med, test_med = train_test_split(test_med, test_size=0.5)

In [13]:
def tokenize_dataset(dataset):
    tokenized_texts = []
    for text in tqdm(dataset['combined'].tolist(), desc="Tokenizing"):
        text = text.lower()
        tokenized_texts.append(tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt'))
    return tokenized_texts

In [14]:
tokenized_train = tokenize_dataset(train_med)
tokenized_val = tokenize_dataset(val_med)
tokenized_test = tokenize_dataset(test_med)

Tokenizing: 100%|██████████| 15000/15000 [00:12<00:00, 1194.76it/s]


In [15]:
class BioGPTDataset(Dataset):
    def __init__(self, encodings, tokenizer):
        self.encodings = encodings
        self.pad_token_id = tokenizer.pad_token_id

    def __getitem__(self, idx):
        # Retrieves the input_ids and attention_mask for the given index
        item = {key: torch.squeeze(val) for key, val in self.encodings[idx].items()}
        item['labels'] = torch.cat([item['input_ids'][1:], torch.tensor([self.pad_token_id])])
        return item

    def __len__(self):
        # Returns the length of the input_ids
        return len(self.encodings)

def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.stack([item["labels"] for item in batch])
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [25]:
train_dataset = BioGPTDataset(tokenized_train, tokenizer)
val_dataset = BioGPTDataset(tokenized_val, tokenizer)
test_dataset = BioGPTDataset(tokenized_test, tokenizer)

In [ ]:
# Training Arguments and Trainer Initialization
training_args = TrainingArguments(
        output_dir='./results_bis',
        num_train_epochs=2,
        per_device_train_batch_size=10,
        per_device_eval_batch_size=10,
        warmup_steps=3000,
        weight_decay=0.01,
        learning_rate= 1e-4,
        logging_dir='./logs',
        logging_steps=6000,
        evaluation_strategy="steps",
        save_steps=6000,
        load_best_model_at_end=True
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
6000,0.910000,0.729985
12000,0.732600,0.696220
18000,0.691300,0.681039


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=23866, training_loss=0.7548726237770259, metrics={'train_runtime': 8387.2826, 'train_samples_per_second': 28.454, 'train_steps_per_second': 2.845, 'total_flos': 6.2357866020864e+16, 'train_loss': 0.7548726237770259, 'epoch': 2.0})

**Testing**

#### bioGPT

In [17]:
model_path = 'bioGPT'
tokenizer_bio = AutoTokenizer.from_pretrained('microsoft/biogpt')
bio_gpt = AutoModelForCausalLM.from_pretrained(model_path).to(device)

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

In [18]:
def generate_answer(model,tokenizer, question, length_answer):
    encoded = tokenizer.encode(question, return_tensors='pt').to(device)
    output = model.generate(
        encoded,
        max_length=length_answer,
        no_repeat_ngram_size=1,
        early_stopping=True,
        num_beams=5
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)[len(question) - 1:]

In [20]:
generated_answers = [generate_answer(bio_gpt,tokenizer_bio, row['question'], len(row['answer'])) for idx,row in tqdm(test_med[:30].iterrows(), total=len(test_med))]


  0%|          | 30/15000 [03:02<25:14:48,  6.07s/it]


#Bert Score

In [22]:
from bert_score import score

# Calculating BERTScore for each pair of generated and reference answers
P, R, F1 = score(generated_answers, test_med[:30]['answer'].tolist(), lang="en")

# Calculating the average of the F1 scores
average_score = F1.mean().item()

print(average_score)

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.7838001251220703
